# About:
First thing i want to mention is that the notebook is not complete yet! most part not gonna changed but improvement is under the construction! Anyway i create the notebook base on kaggle micro courses from python till feature engineering, reading some blog posts from toward data science(all are mentioned at reference links), kaggle(mentioned too) and roaming in scikit learn APIs and LightGB documents. so if you want to get familiar with EDA, first part is for you (mostly data analysis works covering but not complete yet!). if you want to go further part 2 is covered machine learning task with building 3 hot regression models, and a light gradient boosting regressor with simple coding in beginner level, and even if you are ok with feature engineering part 3 is the thing for you!(some part are lack of suitable description that in next weeks they'll be complete.)
hope you find it valuable.

# Introduction:
We are going through an inevitable concept in every data science project that is "Exploaratory Data Analysis" aka "EDA".
I'm not going to extend it so much as i prepare some useful references at the end of this notebook, but just in case to get familiar with the following cells, you should know EDA is an approach or philosophy for data analysis that employs a variety of techniques (mostly graphical) to make the part of your analysis work more reasonable and prepare insights for machine learning part(for more detail check The first reference link at the end of The notebook)
In the middle of our work, we will goinig to do machine learning task with 4 model in beginner level.
At last ending up our work with using of feature engineering concepts for improving and boosting our model.

# 1.EDA

# 1.1.Import Required Modules

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
import lightgbm as lgb
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder 
import sklearn.metrics
from sklearn.metrics import mean_absolute_error
import category_encoders as ce
from sklearn.decomposition import TruncatedSVD
import itertools
from sklearn.feature_selection import SelectKBest, f_regression

# 1.2.Loading data to df (DataFrame!)

In [ ]:
nyc_filepath = '/kaggle/input/nyc-property-sales/'
os.listdir(nyc_filepath)

In [ ]:
df = pd.read_csv(nyc_filepath + 'nyc-rolling-sales.csv')

In [ ]:
df.head()

# 1.3.Checking the types of data

In [ ]:
types = list()
for idx, col in enumerate(df.columns):
    dtype = str(df[col].dtype)
    types.append([col, dtype, idx])
    
print('[COLUMN NAME, '+'COLUMN DATA TYPE, '+'COLUMN POSITION IN DATA FRAME]') 
types

# 1.4.Dropping unnecessary columns!( if they're already exist!)

In [ ]:
# First thing that seems utterly inutile! is 'Unnamed: 0' column
df.drop(['Unnamed: 0'], axis=1, inplace=True)

# Second one was 'EASE-MENT', cause it is full empty column!
df.drop(['EASE-MENT'], axis=1, inplace=True)

# Third one for this notebook is 'SALE TIME'. it's truely an important Feature if we would to trace sale dates!
df.drop(['SALE DATE'], axis=1, inplace=True)

# Fourth one is 'APARTMENT NUMBER'. not going to use of it!
df.drop(['APARTMENT NUMBER'], axis=1, inplace=True)

# Fifth one is 'BUILDING CLASS CATEGORY' becuase we have a better of it named 'BUILDING CLASS AT PRESENT'!
df.drop(['BUILDING CLASS CATEGORY'], axis=1, inplace=True)

# Sisxth one is 'BUILDING CLASS AT PRESENT' as same reason as above one, but i'm not sure yet in next version it could be not drop!
df.drop(['BUILDING CLASS AT PRESENT'], axis=1, inplace=True)

# Seventh one is 'TAX CLASS AT PRESENT', not sure again it could be a positive result in our model in next version i decide about it!
df.drop(['TAX CLASS AT PRESENT'], axis=1, inplace=True)

now we can see the modified data frame after 7 columns dropped!

In [ ]:

df.head()

# 1.5.Renaming columns and index if we have better names for them!

in following cells i'm going to fix index of data frame with help of 'ADDRESS' column

In [ ]:
df.ADDRESS.describe()

In [ ]:
# making index for our df with one of it's column called 'ADDRESS'! 
# in below code we added new column named 'Property' at the end of our data frame
df['Property'] = [idx for idx in range(df.ADDRESS.count())]

# here we fixed the index issue!
df.index = df['Property']

# aftre that we just drop the added column from end of our data frame!
df = df.drop(['Property'], axis=1)

print('Now it seems we have a neat dataframe so far!')
df.head()

# 1.6.Trying to find duplicate rows and drop them! (if they're already exist!)

In [ ]:
# Investigating for Duplicated rows!
print(f"Original dataframe shape: {df.shape}")

duplicate_rows = df[df.duplicated()]
print(f"Duplicated rows: {duplicate_rows.shape}")

# Dropping the duplicates 
df = df.drop_duplicates()
print('Modified dataframe shape: {}'.format(df.shape))

# 1.7.Detecting missing values!

In [ ]:
def Detecting_missing_vals(data):
    total = data.isnull().sum()
    percent = (data.isnull().sum() / data.isnull().count() * 100)
    tpt = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    
    types=list()
    for col in data.columns:
        dtype = str(data[col].dtype)
        types.append(dtype)
        
    tpt['Types'] = types
    
    return(np.transpose(tpt))

In [ ]:
Detecting_missing_vals(df)

It seems likely strange feeling which as a 'Data Scientist' you getting a data set without any missing values!

In [ ]:
# By the way this is another tool that pandas gives it to you for checking missing values!
df.isnull().sum()

# 1.8.Detecting and Hunting Outliers with plotting tools and statistical concepts respectively!

In statistics, an outlier is an observation point that is distant from other observations.
The above definition suggests that outlier is something which is separate or different from the crowd.(more detail in reference link)

In descriptive statistics, a box plot is a method for graphically depicting groups of numerical data through their quartiles. Box plots may also have lines extending vertically from the boxes (whiskers) indicating variability outside the upper and lower quartiles, hence the terms box-and-whisker plot and box-and-whisker diagram. Outliers may be plotted as individual points.
Above definition suggests, that if there is an outlier it will plotted as point in boxplot but other population will be grouped together and display as boxes. Let’s try and see it ourselves.

In [ ]:
sns.boxplot(x=df['BLOCK'])

you can see those points beyond the about 13,800... they're outliers or noise, in the next weeks i'm going to extend them much more!

from now on we track the 'SALE PRICE' column in three following cells and try to do some changes!

In [ ]:
df['SALE PRICE']

here i'm replacing '-' with 0, as i had no idea about those values i decided to replace them with just bunch of zeros!
but we could fill those values with the mean of the SALE PRICE column, In next version of this notebook i'm going to make the right decision for them! don't worry

In [ ]:
df['SALE PRICE'].replace(to_replace='-', value=0, regex=True, inplace=True)

Here i change the type of data in 'SALE PRICE' column

In [ ]:
df['SALE PRICE'] = df['SALE PRICE'].astype('float64')

Finding Outliers with 'Interquartile Range' score or IQR score! (midspread, middle 50% and H_spread are the other names!)
Simply it differences lower 25% from upper 75% and returns the 50%(between upper and lower quartiles) or in other words central value's dispersion of the features.
It measures like the Standard deviation and Variance but in a more robust way!

In [ ]:
Q1 = df.quantile(0.25) # first quartile
Q3 = df.quantile(0.75) # third quartile
IQR = Q3 - Q1
print("We can see central value of our columns:\n")
print(IQR)

In [ ]:
# Original df rows
df.shape[0]

In [ ]:
# Modified df rows
df = df[ ~( (df < (Q1 - 1.5 * IQR) ) | (df > (Q3 + 1.5 * IQR) ) ).any(axis=1)]
df.shape[0]

In [ ]:
# Because of IQR outlier detection technique we gave a messy index for rows!
df.index

In [ ]:
# Reindex data set as well as shown in below!
df.index = [idx for idx in range(df.ADDRESS.count())]
df.index

# 1.9.Plotting and Explorating the data

we are depicting some concepts and information about data with lovely plots!

in below we're plotting the number of properties that are belonging to each five boroughs

In [ ]:
sns.countplot(df['BOROUGH'] ,  palette='Set3')

listing five new york boroughs for iterating in following after that

In [ ]:
Five_boroughs = ["Manhattan", "Bronx", "Brooklyn", "Queens", "Staten Island"]

for idx, bor in zip(df['BOROUGH'].unique(), Five_boroughs):
    print('There are #{} Properties belong to {} Borough.'.format( df[ df['BOROUGH'] == idx].shape[0], bor ),'\n' )

In [ ]:
# I just went a step further for percentage of properties belong to each borough!
for idx, bor in zip(df['BOROUGH'].unique(), Five_boroughs):
    print("{:.2f}% of Properties belong to {} Borough."\
          .format( 100 * df[ df['BOROUGH'] == idx ].shape[0] / df.shape[0] , bor), '\n'  )

Baking a "Pie" Based on Preceding Cells !

In [ ]:
Boroughs = ['Manhattan', 'Bronx', 'Brooklyn', 'Queens', 'Staten Island']

Properties_Per_Each_Borough = [10751, 5196, 16201, 21252, 7407] # we already had them!

colors = ['#BE61CA', '#7CDDDD', 'yellowgreen', 'orange', 'purple'] # based on https://www.nycgo.com/neighborhoods-boroughs/about-nyc-five-boroughs/

explode = (0.1, 0, 0.1, 0.1, 0)

plt.pie(Properties_Per_Each_Borough, explode= explode, labels = Boroughs, colors=colors,\
        startangle=90, autopct= '%1.1f%%', shadow=True)
    
print("\nPlotting Pie Chart:")
plt.title('Proportioning of Properties Per each Borough')

plt.axis('equal')

plt.legend(['Manhat', 'Bronx', 'Brooklyn', 'Queens', 'Staten Island'],
           loc='upper right')
plt.show()

In [ ]:
# From now on within two or three cells in following trying to get some question's answers!
prices = 0
for price in df['SALE PRICE']:
    prices += price
    
AVG_PRICE = prices / len(df['SALE PRICE'])
print('Average price of Properties Sale Prices is: {:.2f}$'.format(AVG_PRICE))

In [ ]:
print("There are {:.2f}% of Properties with more than half of the Average Price!"\
      .format(100 * df[ df['SALE PRICE'] > 478475.24 ].shape[0] / df.shape[0] )  )

we could get average(mean) price and other stats features with pandas built_in methods as shown in below!

In [ ]:
max_price = df['SALE PRICE'].max()
min_price = df['SALE PRICE'].min()
count = df['SALE PRICE'].count()
mean_price = df['SALE PRICE'].mean()
print('max_price: {}$\nmin_price: {}$\ncount: {}\nmean_price: {:.2f}$'.format(max_price, min_price, count, mean_price))

Preparing for settle in "Rise of Modelling Age" with some basic information gathering about our numerical and categorical data!

In [ ]:
categorical_cols = [col for col in df.columns if df[col].dtype == 'object']
numerical_cols = [col for col in df.columns if df[col].dtype in ['int64', 'float64']]

In [ ]:
categorical_cols

I suprised as much as you when saw 'LAND SQUARE FEET' and 'GROSS SQUARE FEET' in categorical features!

In [ ]:
df['LAND SQUARE FEET']

Familiar pattern used for getting rid of '-' values!

In [ ]:
# this part could slightly changed in next version of the notebook
df['LAND SQUARE FEET'].replace(to_replace='-', value=0, regex=True, inplace=True)
df['LAND SQUARE FEET'] = df['LAND SQUARE FEET'].astype('int64')
df['LAND SQUARE FEET']

In [ ]:
df['GROSS SQUARE FEET'].replace(to_replace='-', value=0, regex=True, inplace=True)
df['GROSS SQUARE FEET'] = df['GROSS SQUARE FEET'].astype('int64')
df['GROSS SQUARE FEET']

# 2.Machine learning task (Regression models in beginner level!)

Model Processing Without Cross Validation ! (shouldn't account it's result as a considerable one!) 

In [ ]:
X = df.drop(['SALE PRICE'], axis=1)
y = df['SALE PRICE']

In [ ]:
X.shape, y.shape

In [ ]:
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size = 0.8, test_size=0.2, random_state=0)

In [ ]:
X_train_full.shape, X_valid_full.shape, y_train.shape, y_valid.shape

I just redefined them with more compacted names!

In [ ]:
cat_cols = [col for col in X_train_full.columns if X_train_full[col].dtype == 'object']
num_cols = [col for col in X_train_full.columns if X_train_full[col].dtype in ['int64', 'float64']]

In [ ]:
X_train_full['NEIGHBORHOOD'].describe()

In [ ]:
X_train_full['ADDRESS'].describe()

In [ ]:
X_train_full['BUILDING CLASS AT TIME OF SALE'].describe()

Choosing those columns with cardinality less than 250 ( cardinality refers to the number of unique values for each categorical column )

In [ ]:
low_car_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 250  and
                X_train_full[cname].dtype == 'object']

In [ ]:
cols = num_cols + low_car_cols # get numerical and categorical columns together
X_train = X_train_full[cols].copy() # cloning 
X_valid = X_valid_full[cols].copy() # cloning

label encoder in one of the basic categorical encoder that maps a unique number to each categorical data for a column

In [ ]:
lb_enc = LabelEncoder()

label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

for col in low_car_cols:
    label_X_train[col] = lb_enc.fit_transform(X_train[col])
    label_X_valid[col] = lb_enc.transform(X_valid[col])

In three following cells i'm using tree based regressor models!

In [ ]:
%%time
RFR = RandomForestRegressor(n_estimators=170, random_state=0, n_jobs=4)
RFR_Trained = RFR.fit(label_X_train, y_train)
preds = RFR_Trained.predict(label_X_valid)
print('RFR Mean Absolute Error: {:.2f}'.format(mean_absolute_error(y_valid, preds)))

Mean Absolute Error (MAE): measures the average magnitude of the errors in a set of predictions, so here in above cell our random forest regressor can tell the price of a property by around 306k usd error from it's real value!

In [ ]:
%%time
GBR = GradientBoostingRegressor(n_estimators=600, learning_rate=0.055, random_state=0)
GBR_Trained = GBR.fit(label_X_train, y_train)
preds = GBR.predict(label_X_valid)
print('GBR Mean Absolute Error: {:.2f}'.format(mean_absolute_error(preds, y_valid)))

Remind that there is a trade off between n_estimators and learning_rate according to scikit learn documents!

In [ ]:
%%time
XGBR = XGBRegressor(n_estimators=700, n_jobs=6)

XGBR_Trained = XGBR.fit(label_X_train, y_train,
                        early_stopping_rounds=20,
                        eval_set=[(label_X_valid, y_valid)],
                        verbose=False)

predictions = XGBR.predict(label_X_valid)
print("XGBR Mean Absolute Error: {:.2f}".format((mean_absolute_error(predictions, y_valid))))

Model Processing With Cross Validation! (More Real and Calculable Results!)

From now on we are trying to give this chance to our model to facing with and learning from all of the data set each time as a
subset part of the whole with cross validation concept, so we can trust more to the result of this kind of model processing
than we just met before!

In [ ]:
X_cross_valid = X.copy()
y_cross_valid = y.copy()

enc = LabelEncoder()
X_cross_valid[low_car_cols] = X_cross_valid[low_car_cols].apply(enc.fit_transform)
X_cross_valid = X_cross_valid[num_cols].join(X_cross_valid[low_car_cols])

our model is going to take five experiments based on 'cv' parameter we set to 5 in below! each time one part is gonna be a validation set and all other gonna be train sets... and each time we have a result from our model.. at the end we can see the average of those five experiments!

In [ ]:
%%time
RF_scores = -1 * cross_val_score(RFR_Trained, X_cross_valid, y_cross_valid,
                                 cv=5,
                                 scoring='neg_mean_absolute_error')

print("Random Forest Regressor cross validation MAE scores:\n",RF_scores)
print(f"Average Random Forest MAE score (across experiments):\n{RF_scores.mean():.2f}")

The performance of our model get worse than before it's considerable but this way we can more trust to our model's performance!

In [ ]:
%%time
GB_scores = -1 * cross_val_score(GBR_Trained, X_cross_valid, y_cross_valid,
                                 cv=5,
                                 scoring='neg_mean_absolute_error')

print("Gradient Boosting cross validation MAE scores:\n",GB_scores)
print(f"Average Gradient Boosting MAE score (across experiments):\n{GB_scores.mean():.2f}")

In [ ]:
%%time
XGB_scores = -1 * cross_val_score(XGBR_Trained, X_cross_valid, y_cross_valid,
                                  cv=5,
                                  scoring='neg_mean_absolute_error')

print("Extended Gradient Boosting cross validation MAE scores:\n",XGB_scores)
print(f"Average XGB MAE score (across experiments):\n{XGB_scores.mean():.2f}")

LightGB models are fascinating! So Lets Do it together!

In [ ]:
cat_features = cat_cols

In [ ]:
base_encoder = LabelEncoder()
base_encoded = df[cat_features].apply(base_encoder.fit_transform)
base_encoded.head()

In [ ]:
num_features = ['BOROUGH', 'BLOCK', 'LOT', 'ZIP CODE','RESIDENTIAL UNITS', 'COMMERCIAL UNITS', 'TOTAL UNITS',\
                'LAND SQUARE FEET', 'GROSS SQUARE FEET', 'YEAR BUILT', 'TAX CLASS AT TIME OF SALE', 'SALE PRICE']
df1 = df.copy()
df1 = df1[num_features].join(base_encoded) # here we have a data frame with numerical and encoded categorical features together

In [ ]:
# I'm going to use of slicing for splitting data with 80, 10 and 10 percent for train, valid and test data sets respectively!

valid_fraction = 0.1
valid_size = int(len(df1) * valid_fraction)

train = df1[:-2 * valid_size] # 80% of data!

valid = df1[-2 * valid_size:-valid_size] # 10% of what is remained!

test = df1[-valid_size:] # last 10% of that!

Your Promissing Land! in Modelling
For more information about LightGBs check this out! (https://lightgbm.readthedocs.io/en/latest/Parameters.html)

In [ ]:
%%time
feature_cols = train.columns.drop('SALE PRICE')

dtrain = lgb.Dataset(train[feature_cols], label=train['SALE PRICE'])
dvalid = lgb.Dataset(valid[feature_cols], label=valid['SALE PRICE'])


params = {
        'boosting_type': 'gbdt',
        'objective': 'mae',
        'metric': 'rmse',
        'max_depth':7,
        'learning_rate': 0.2555,
        'num_leaves': 31,
        'verbose': 1, 
        }

n_estimators = 100

bst = lgb.train(params, dtrain, n_estimators, valid_sets=[dvalid],\
                early_stopping_rounds=15, verbose_eval=True)

In [ ]:
%%time
test_pred = bst.predict(test[feature_cols])
test_score = mean_absolute_error(test['SALE PRICE'], test_pred)

print(f"Test MAE: {test_score:.2f}")

It is amazing that we get a considerable! improving as the error for our Model! first model get around 306k usd error for a property sale price but this one is more better than that!

# 3.Feature Enginnering:

# 3.1.Baseline Model


The combination of borough, block, and lot forms a unique key for property in New York City. Commonly called a "BBL".

In [ ]:
 df = df.assign(BBL = df['BOROUGH'] + df['BLOCK'] + df['LOT'])

In [ ]:
df.head()

In [ ]:
cat_features = cat_cols
cat_features

In [ ]:
numeric_features = df.columns.drop(cat_features)
numeric_features

In [ ]:
encoder = LabelEncoder()
encoded = df[cat_features].apply(encoder.fit_transform)
encoded.head()

In [ ]:
data = df[numeric_features].join(encoded)
data.head()

In [ ]:
valid_fraction = 0.1
valid_size = int(len(data) * valid_fraction)

train = data[:-2 * valid_size]

valid = data[-2 * valid_size:-valid_size]

test = data[-valid_size:]

in cell below we are trace back the target distribution in train, valid and test sets respectively!

In [ ]:
for each in [train, valid, test]:
    print(f"SALE PRICE mean dispersion = {each['SALE PRICE'].mean():.2f}")

In [ ]:
feature_cols = train.columns.drop('SALE PRICE')

dtrain = lgb.Dataset(train[feature_cols], label=train['SALE PRICE'])
dvalid = lgb.Dataset(valid[feature_cols], label=valid['SALE PRICE'])

params = {
        'boosting_type': 'gbdt',
        'objective': 'mae',
        'metric': 'rmse', 
        'learning_rate': 0.2555,
        'num_leaves': 31,
        'max_depth':7,
        'verbose': 0, 
        }
    
n_estimators = 100

bst = lgb.train(params, dtrain, n_estimators, valid_sets=[dvalid],\
                early_stopping_rounds=15, verbose_eval=False)

In [ ]:
ypred = bst.predict(test[feature_cols])
score = mean_absolute_error(test['SALE PRICE'], ypred)

print(f"Test MAE: {score:.2f}")

# 3.2.Categorical Encodings:
Label Encoding:

In [ ]:
baseline_data = data[numeric_features].join(encoded)

we are just rewriting some of our previous work but more concrete!

In [ ]:
def get_data_splits(dataframe, valid_fraction=0.1):
    valid_fraction = 0.1
    valid_size = int(len(dataframe) * valid_fraction)

    train = dataframe[ :-2 * valid_size]
    # valid size == test size, last two sections of the data
    valid = dataframe[-2 * valid_size : -valid_size]
    test = dataframe[-valid_size:]
    
    return train, valid, test

In [ ]:
def train_model(train, valid):
    
    feature_cols = train.columns.drop('SALE PRICE')

    dtrain = lgb.Dataset(train[feature_cols], label=train['SALE PRICE'])
    dvalid = lgb.Dataset(valid[feature_cols], label=valid['SALE PRICE'])

    params = {
            'boosting_type': 'gbdt',
            'objective': 'mae',
            'metric': 'rmse',
            'max_depth': 7,
            'num_leaves':31,
            'learning_rate': 0.2555,
            'verbose': 0, 
            }

    n_estimators = 100

    bst = lgb.train(params, dtrain, n_estimators, valid_sets=[dvalid],\
                    early_stopping_rounds=15, verbose_eval=False)
    
    valid_pred = bst.predict(valid[feature_cols])
    valid_score = mean_absolute_error(valid['SALE PRICE'], valid_pred)
    
    print(f"Validation MAE score: {valid_score:.2f}")
    return bst

We focused on Model validation stage Error!

In [ ]:
# Training a model on the baseline data
train, valid, _ = get_data_splits(baseline_data)
bst = train_model(train, valid)

Count Encoding:

In [ ]:
# Count Encoding
cat_features = ['NEIGHBORHOOD', 'ADDRESS', 'BUILDING CLASS AT TIME OF SALE']

count_enc = ce.CountEncoder()

count_encoded = count_enc.fit_transform(df[cat_features])

data = baseline_data.join(count_encoded.add_suffix("_count"))

# Training a model on the baseline data
train, valid, test = get_data_splits(data)
Model_CouEnc = train_model(train, valid)

we got worse result!

Target Encoding (Regression Task!):

In [ ]:
# Target Encoding
cat_features = ['NEIGHBORHOOD', 'ADDRESS', 'BUILDING CLASS AT TIME OF SALE']

# Create the encoder itself
target_enc = ce.TargetEncoder(cols=cat_features)

train, valid, _ = get_data_splits(data)

# Fit the encoder using the categorical features and target, so we avoid data(target) leakage!
target_enc.fit(train[cat_features], train['SALE PRICE'])

# Transform the features, rename the columns with _target suffix, and join to dataframe
train = train.join(target_enc.transform(train[cat_features]).add_suffix('_target'))
valid = valid.join(target_enc.transform(valid[cat_features]).add_suffix('_target'))

Model_TarEnc = train_model(train, valid)

worse result than two preceding cells!!

CatBoost Encoding (another kind of Target Encoding!) :

In [ ]:
# CatBoost Encoding
cat_features = ['NEIGHBORHOOD', 'ADDRESS', 'BUILDING CLASS AT TIME OF SALE']

target_enc = ce.CatBoostEncoder(cols=cat_features)

train, valid, _ = get_data_splits(data)

# Fit the encoder using the categorical features and target, so we avoid data(target) leakage!
target_enc.fit(train[cat_features], train['SALE PRICE'])

# Transform the features, rename the columns with _target suffix, and join to dataframe
train = train.join(target_enc.transform(train[cat_features]).add_suffix('_cb'))
valid = valid.join(target_enc.transform(valid[cat_features]).add_suffix('_cb'))

Mode_CatBoost = train_model(train, valid)

i don't know how should i explain these crazy results, but if you had one just let me know! (by the way i'm not professional!)

you're going to see a spectacular encoder! just watch this beast and it's performance!

Reference Link! https://www.kaggle.com/matleonard/encoding-categorical-features-with-svd

In [ ]:
# Singular Value Decomposition (SVD)
svd = TruncatedSVD(n_components=3)

train, valid, _ = get_data_splits(baseline_data)

# Create a sparse matrix with cooccurence counts
pair_counts = train.groupby(['NEIGHBORHOOD', 'BUILDING CLASS AT TIME OF SALE'])['SALE PRICE'].count()
pair_counts.head(10)

In [ ]:
# create a data frame with .unstack method
pair_matrix = pair_counts.unstack(fill_value=0)
pair_matrix

In [ ]:
svd_encoding = pd.DataFrame(svd.fit_transform(pair_matrix))
svd_encoding.head()

In [ ]:
encoded = svd_encoding.reindex(baseline_data['NEIGHBORHOOD']).set_index(baseline_data.index)
encoded.head()

In [ ]:
# Join encoded feature to the dataframe, with info in the column names
data_svd = baseline_data.join(encoded.add_prefix("NEIGHBORHOOD_BUILDING CLASS_SVD_"))
data_svd.head()

In [ ]:
train, valid, _ = get_data_splits(data_svd)
SVD_model = train_model(train, valid)

it was better than preceding nightmares!

# 3.3.Feature Generation:

Interactions : as kaggle instructor mentioned it is just combination of numerical features together as new feature feeding for our model 

In [ ]:
# Interactions
interactions = pd.DataFrame(index = df.index)

for col1, col2 in itertools.combinations(cat_features, 2):
    new_col_name = '_'.join([col1, col2])
    # Convert to strings and combine
    new_values = df[col1].map(str) + "_" + df[col2].map(str)
    label_enc = LabelEncoder()
    interactions[new_col_name] = label_enc.fit_transform(new_values)
    
baseline_data = baseline_data.join(interactions)

In [ ]:
baseline_data.head()

In [ ]:
train, valid, _ = get_data_splits(baseline_data)
FeatGen_model = train_model(train, valid)

we still can hold out our hope

# 3.4.Feature Selection:

Univariate Feature Selection : The simplest and fastest methods are based on univariate statistical tests. For each feature, measure how strongly the target depends on the feature using a statistical test like  χ2 (chi_square)  or ANOVA.

With SelectKBest, we define the number of features to keep, based on the score from the scoring function. Using .fit_transform(features, target) we get back an array with only the selected features.

In [ ]:
feature_cols = baseline_data.columns.drop('SALE PRICE')

# Keep Top 5 features
selector = SelectKBest(f_regression, k=5)
X_new = selector.fit_transform(baseline_data[feature_cols], baseline_data['SALE PRICE'])
X_new

In [ ]:
# due to leakage issue we do it again but this time using of train set after data Splitting!
feature_cols = baseline_data.columns.drop('SALE PRICE')
train, valid, _ = get_data_splits(baseline_data)

# Keep 5 features
selector = SelectKBest(f_regression, k=5)

X_new = selector.fit_transform(train[feature_cols], train['SALE PRICE'])
X_new

we can use .inverse_transform to get back an array with the shape of the original data.
so now we can trace back of all rejected columns by their main charactrestic which is 0 values!

In [ ]:
selected_features = pd.DataFrame(selector.inverse_transform(X_new), 
                                 index=train.index, 
                                 columns=feature_cols)
selected_features.head()

A DataFrame with the same index and columns as the training set,
but all the dropped columns are filled with zeros,so
non_zero variance is our margin! for detecting Top 5 best Features from the others!

In [ ]:
selected_columns = selected_features.columns[selected_features.var() != 0]

# Get the valid dataset with the selected features.
valid[selected_columns].head()

I'm going to use of data_svd(best encoded data among others we saw!) to splitting and then take those best features that just selected based
on univariate stat test with hope of better result!

In [ ]:
train, valid, _ = get_data_splits(data_svd)

train = train[selected_columns].join(df['SALE PRICE'])
valid = valid[selected_columns].join(df['SALE PRICE'])

model_B5F = train_model(train, valid)

L1 regularization Feature Selection (for Regression Task!) :

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

train, valid, _ = get_data_splits(baseline_data)

X, y = train[train.columns.drop("SALE PRICE")], train['SALE PRICE']

# I didn't pass any parameter, so it is as naive as it can be!
lasso = Lasso().fit(X, y)
model = SelectFromModel(lasso, prefit=True)

X_new = model.transform(X)
X_new

In [ ]:
# Get back the kept features as a DataFrame with dropped columns as all 0s
selected_features = pd.DataFrame(model.inverse_transform(X_new), 
                                 index=X.index,
                                 columns=X.columns)

# Dropped columns have values of all 0s, keep other columns 
selected_columns = selected_features.columns[selected_features.var() != 0]
selected_columns

# Reference Links: 

# EDA explanation and example
# https://www.itl.nist.gov/div898/handbook/eda/section1/eda11.htm
# https://towardsdatascience.com/exploratory-data-analysis-in-python-c9a77dfa39ce
# https://towardsdatascience.com/ways-to-detect-and-remove-the-outliers-404d16608dba
# https://www.kaggle.com/gpreda/santander-eda-and-prediction
# Categorical Encoding Techniques_ SVD
# https://www.kaggle.com/matleonard/encoding-categorical-features-with-svd